### (Short) EDA of bids data and Upload to DB

- download units and permits
- extract Anlagen EEG from open mastr dump
- units-table: How do the mastr nrs start? Are units starting with A among the units, as they are in earlier years of the bid-data?
- can all units with a Anlagen_Registernr be linked to units of the units-table?
- can any columns be omitted? Following Database Design rules, administrative/locational data should not be kept in the the bid data if they can be linked to via foreign key to the units data
- If they can be linked, delete the data in the bids-table -> since this will not be updated

In [ ]:
### Download, Inspect and Upload Permit Data
import pandas as pd
import psycopg2
import os
import numpy as np
from dotenv import load_dotenv
from pandas.api.types import is_datetime64_any_dtype
from datetime import datetime
import time
import random
import re
import pickle
import sqlalchemy
from open_mastr import Mastr

In [ ]:
### Get Units data

# Load environment variables from .env file
load_dotenv()

# Get connection parameters from environment variables
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
ssl_cert_path = os.getenv("SSL_CERT_PATH")

# Construct the connection string
conn_str = f"dbname={dbname} user={user} password={password} host={host} port={port} sslmode=require sslrootcert={ssl_cert_path}"

# Etablish connection object

#conn.close()
conn = psycopg2.connect(conn_str)

sql_select = "SELECT * FROM public.wind_extended;"
df_wind = pd.read_sql(sql_select, conn)

In [ ]:
### Get bid data
with open("../data/mastr_bids/bids_cleaned_2017_2023.pkl", mode = "rb") as pkl_file:
   df_bids_all = pickle.load(pkl_file)

In [ ]:
def find_unique_starting_letters(x):
    x_2 = x.fillna('')
    series_match = x_2.apply(lambda ser: re.findall(r"^[A-Za-z]+", ser))

    vals = []
    for match_re in series_match.values:
        if len(match_re) == 1:
            vals.append(match_re[0])

    return pd.unique(vals).tolist()

In [ ]:
### How do mastr_nr s in df_wind and Anlagen_Registernr in df_bids_all start?
cols_nr = [col for col in df_wind.columns if re.search("mastr_nummer", col)]

cols_nr_start = {}

### Find all beginning unique Letters in the mastr_nummer columns 
for col in cols_nr:
    cols_nr_start[col] = find_unique_starting_letters(df_wind[col])

# List to store tuples, where each tuple represents one row for the long df    
rows_list = []

# loop through each nr-column
for col, values in cols_nr_start.items():
    # loop through each unique value of the number column and create tuple with column name in it
    for value in values:
        rows_list.append((col, value))

pd.DataFrame(rows_list, columns = ["mastr-column", "starting-letters"])

In [ ]:
find_unique_starting_letters(df_bids_all["Register_Anlagennr"])

### Discrepancy in df_bids identifiers

- No foreign key nr in wind_extended start with only A
- Try: Can they be linked to the eeg_anlagen table?
- Try: When A in df_bids.Anlagen_Registernr is exchanged by SEE -> can it be linked to the units table

In [ ]:
# Try: When A in df_bids.Anlagen_Registernr is exchanged by SEE -> can it be linked to the units table

# copy df
df_bids_a = df_bids_all.copy()
# .str is an acessor used to apply simple string-methods (used on one string object "foo") on a series
ind = df_bids_a["Register_Anlagennr"].fillna("").str.startswith("A")
df_bids_a = df_bids_a[ind]
df_bids_a = df_bids_a[['Name des Bieters', 
       'Landkreis', 'Postleitzahl', 'Gemeinde', 'Gemarkung',
       'Flur / Flurstück', 'Register_Anlagennr', 'Gebotsdatum', 'Zuschlags-Nr']]
# Create column A -> SEE
df_bids_a["A_to_SEE"] = df_bids_a["Register_Anlagennr"].str.replace("A", "SEE")
# Create column firs three positions -> SEE
df_bids_a["pos_3_to_SEE"] = df_bids_a["Register_Anlagennr"].str.replace(r"^.{3}", "SEE")

# Create column A -> SEE
df_bids_a["A_to_EEG"] = df_bids_a["Register_Anlagennr"].str.replace("A", "EEG")
# Create column firs three positions -> SEE
df_bids_a["pos_3_to_EEG"] = df_bids_a["Register_Anlagennr"].str.slice_replace(0,3, "EEG")

In [ ]:
rows_list = []
for key, row in df_wind.iterrows():
    rows_list.append((row["einheit_mastr_nummer"], 
    row["einheit_mastr_nummer"] in df_bids_a["A_to_SEE"].tolist(),
    row["einheit_mastr_nummer"] in df_bids_a["pos_3_to_SEE"].tolist(),
    row["eeg_mastr_nummer"] in df_bids_a["A_to_EEG"].tolist(),
    row["eeg_mastr_nummer"] in df_bids_a["pos_3_to_EEG"].tolist()))

df_test = pd.DataFrame(rows_list, columns=["einheit_mastr_nr", "A_to_SEE", "pos_3_to_SEE", "A_to_EEG", "pos_3_to_EEG"])

In [ ]:
for col in ["A_to_SEE", "pos_3_to_SEE", "A_to_EEG", "pos_3_to_EEG"]:
    print(col, df_test[col].sum())

In [ ]:
### Can the registernr at least be found?
# copy df
df_bids_see = df_bids_all.copy()
# .str is an acessor used to apply simple string-methods (used on one string object "foo") on a series
ind = df_bids_see["Register_Anlagennr"].fillna("").str.startswith("SEE")
df_bids_see = df_bids_see[ind]
df_bids_see = df_bids_see[['Name des Bieters', 
       'Landkreis', 'Postleitzahl', 'Gemeinde', 'Gemarkung',
       'Flur / Flurstück', 'Register_Anlagennr', 'Gebotsdatum', 'Zuschlags-Nr']]

rows_list = []
for key, row in df_wind.iterrows():
    rows_list.append((row["einheit_mastr_nummer"], 
    row["einheit_mastr_nummer"] in df_bids_see["Register_Anlagennr"].tolist()))

df_test = pd.DataFrame(rows_list, columns=["einheit_mastr_nr", "Register_Anlagennr"])
print(df_test["Register_Anlagennr"].sum(), len(df_bids_see), len(df_bids_all))

### 2854 unit_mastr_nrs from wind_extended can be found in the 2868 mastr_nrs starting with SEE from df_bids_see of all 4418 bid-units

In [ ]:
### Retrieve the Anlagentable
### Data was already downloaded with open_mastr into local sqllite DB
db = Mastr()
conn = db.engine # Connection engine

tables = pd.read_sql_query('SELECT name from sqlite_master where type= "table";', conn)
df_eeg = pd.read_sql_table("wind_eeg", con=conn)

In [162]:
df_eeg.columns
### Interesting columns with possible link to Anlagennr in Bids:

df_eeg[['Zuschlagsnummer', 'VerknuepfteEinheit', 'AnlagenschluesselEeg',
       'AnlagenkennzifferAnlagenregister',
       'AnlagenkennzifferAnlagenregister_nv']]

### AnlagenkennzifferAnlagenregister seems interesting

,Zuschlagsnummer,VerknuepfteEinheit,AnlagenschluesselEeg,AnlagenkennzifferAnlagenregister,AnlagenkennzifferAnlagenregister_nv
0,None,SEE982417853618,E23572010000000001000024015700004,A5176470103628,False
1,None,SEE913741454097,E3070101Wn00000000000002891900001,A2771450152172,False
2,None,SEE914108319653,E23572010000000001000024015700006,A5214480103600,False
3,None,SEE901069753206,E30944010000000000001044451202340,None,False
4,None,SEE975973981666,E2191501WIXX000000010416384100000,A1735431090380,False
...,...,...,...,...,...
32411,None,SEE943758007153,None,None,False
32412,None,SEE904892741723,None,None,False
32413,None,SEE999728681953,None,None,False
32414,WIN22-1/117,SEE936618774630,None,None,False


In [ ]:
print(df_bids_a["Register_Anlagennr"].isin(df_eeg["AnlagenkennzifferAnlagenregister"]).sum(), len(df_bids_a))
# 529 units in the bids_a table can be linked to the Anlagen-Table via the ANlagennr starting with a

In [ ]:
### How many can be linked to the eeg table via the Zuschlagsnr
len(df_bids_all) # 4418 units
### How do they relate to the Zuschlagsdatum in df_bids_all

### 1. Can all bids be linked via the Zuschlagsnr? Do all have a Zuschlagsnr?
df_bids_all.info() # Zuschlags-Nr 4418 non-null - All have a Zuschlagsnr

df_bids_all["Zuschlags-Nr"].isin(df_eeg["Zuschlagsnummer"]).sum() ### 1392 / 4418

len(df_eeg.Zuschlagsnummer.unique().tolist()) # 1220 unique Zuschlagsnr in df_eeg

len(df_bids_all["Zuschlags-Nr"].unique().tolist()) # 2059 unique Zuschlagsnr in df_bids_all

In [ ]:
df_bids_all["A_SEE_none"] = df_bids_all["Register_Anlagennr"].str.extract(r"^([A-Za-z]).*")

In [ ]:
df_bids_all["Zuschlags-Nr"].apply(lambda x: len(x)).unique() # All Zuschlagsnr of len 11
len_zsnr = df_eeg["Zuschlagsnummer"].fillna("").apply(lambda x: len(x))

In [ ]:
pattern_1 = r'^(WIN\d{2}-\d{1,2}-\d{3})$'
scheme_1 = "1_WIN%Y-%M-nr"
pattern_2 = r'^(WIN\d{2}-\d{1,2}/\d{3})$'
scheme_2 = "2_WIN%Y-%M/nr"

def retrieve_nr_scheme(item):
    if len(item) == 0:
        return "0 -"
    elif re.match(pattern_1, item):
        return scheme_1
    elif re.match(pattern_2, item):
        return scheme_2
    else:
        return "3_X"

df_bids_all.copy()    
df_bids_all["Zuschlags-Nr"].fillna("").apply(lambda x: retrieve_nr_scheme(x)) # '1_WIN%Y-%M-nr', '2_WIN%Y-%M/nr'        
df_eeg["Zuschlagsnummer"].fillna("").apply(lambda x: retrieve_nr_scheme(x)).unique() # '0 -', '2_WIN%Y-%M/nr', '3_X'

# Mismatch in how the Zuschlagsnr are structured
# Replace - with / in Zuschlagsnr in df_bids_all -> then compare merging



In [ ]:
df_bids_all["Zuschlagsnummer"] = df_bids_all["Zuschlags-Nr"].apply(lambda x: re.sub(r"-(\d{3})$", r"/\1", x))
df_bids_all["ZS_nr_in_eeg"] = df_bids_all["Zuschlagsnummer"].isin(df_eeg["Zuschlagsnummer"])
df_bids_all["Anl_nr_in_eeg"] = df_bids_all["Register_Anlagennr"].isin(df_eeg["AnlagenkennzifferAnlagenregister"])
df_bids_all["Mastr_nr_in_eeg"] = df_bids_all["Register_Anlagennr"].isin(df_eeg["VerknuepfteEinheit"])
df_bids_all["A_SEE_none"] = df_bids_all["A_SEE_none"].fillna("x")

In [ ]:
### Overview (by Gebotsdatum)

# Nr of units, Nr of Zuschlagsnr, Nr of Zuschlagsnr found, Nr of See found, Nr of A found, Nr of Zuschlagsnr and See/A found, Nr of Zuschlagsnr found but not See/A. Nr of See/A found but not Zuschlagsnr
group_gebdatum_df_bids_all = df_bids_all.groupby(["Gebotsdatum", "A_SEE_none"])

In [129]:
df_bids_sum = pd.DataFrame()

for name, df_group in group_gebdatum_df_bids_all:
    # Nr of units
    n_units = len(df_group)
    n_zsnr = len(df_group["Zuschlagsnummer"].unique())
    
    # n_zsnr_in_eeg = df_group.groupby("Zuschlagsnr")
    n_zsnr_in_eeg = len(df_group[df_group["ZS_nr_in_eeg"]]["Zuschlagsnummer"].unique())
    
    n_mastr_nr_in_eeg = df_group["Mastr_nr_in_eeg"].sum()
    n_anl_nr_in_eeg = df_group["Anl_nr_in_eeg"].sum()
    
    n_zsnr_unit_nr_in_eeg = 0
    n_only_zsnr_in_eeg = 0
    n_only_unit_nr_in_eeg = 0
    
    for ind, row in df_group.iterrows():
        
        if name[1] == "S":
        
            if row["ZS_nr_in_eeg"] == True and row["Mastr_nr_in_eeg"] == True:
                  n_zsnr_unit_nr_in_eeg += 1
            
            if row["ZS_nr_in_eeg"] == True and row["Mastr_nr_in_eeg"] == False:
                  n_only_zsnr_in_eeg += 1
            
            if row["ZS_nr_in_eeg"] == False and row["Mastr_nr_in_eeg"] == True:
                  n_only_unit_nr_in_eeg += 1
        
        elif name[1] == "A":
            if row["ZS_nr_in_eeg"] == True and row["Anl_nr_in_eeg"] == True:
                  n_zsnr_unit_nr_in_eeg += 1
            
            if row["ZS_nr_in_eeg"] == True and row["Anl_nr_in_eeg"] == False:
                  n_only_zsnr_in_eeg += 1
            
            if row["ZS_nr_in_eeg"] == False and row["Anl_nr_in_eeg"] == True:
                  n_only_unit_nr_in_eeg += 1
    
    dict_row = {"Gebotsdatum": name[0], "n_units": n_units, "n_zsnr":n_zsnr, "type_unit_nr":name[1], 
                    "n_zsnr_in_eeg":n_zsnr_in_eeg, "n_mastr_in_eeg":n_mastr_nr_in_eeg, "n_anl_nr_in_eeg":n_anl_nr_in_eeg,
                    "n_zsnr_unit_nr_in_eeg":n_zsnr_unit_nr_in_eeg,
                    "n_only_zsnr_in_eeg":n_only_zsnr_in_eeg, 
                    "n_only_unit_nr_in_eeg":n_only_unit_nr_in_eeg}
    
    if df_bids_sum.empty:
          df_bids_sum = pd.DataFrame(dict_row, index = [0])
    else:
          df_bids_sum = pd.concat([df_bids_sum, pd.DataFrame(dict_row, index = [0])], ignore_index=True)


In [130]:
df_bids_sum.sort_values(by=["type_unit_nr", "Gebotsdatum"])

,Gebotsdatum,n_units,n_zsnr,type_unit_nr,n_zsnr_in_eeg,n_mastr_in_eeg,n_anl_nr_in_eeg,n_zsnr_unit_nr_in_eeg,n_only_zsnr_in_eeg,n_only_unit_nr_in_eeg
3,2018-02-01,215,83,A,60,0,133,114,19,19
4,2018-05-01,187,111,A,93,0,108,107,51,1
5,2018-08-01,200,86,A,78,0,131,131,49,0
6,2018-10-01,113,57,A,49,0,51,51,37,0
7,2019-02-01,133,67,A,63,0,92,92,35,0
8,2019-05-01,8,4,A,4,0,5,5,3,0
10,2019-08-01,2,1,A,1,0,2,2,0,0
12,2019-09-01,2,2,A,2,0,2,2,0,0
14,2019-10-01,4,1,A,1,0,4,4,0,0
16,2019-12-01,7,3,A,3,0,1,1,6,0


In [155]:
df_eeg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32416 entries, 0 to 32415
Data columns (total 26 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   PrototypAnlage                                 25627 non-null  object        
 1   PilotAnlage                                    5639 non-null   object        
 2   InstallierteLeistung                           32416 non-null  float64       
 3   VerhaeltnisErtragsschaetzungReferenzertrag     2456 non-null   float64       
 4   VerhaeltnisReferenzertragErtrag5Jahre          12987 non-null  float64       
 5   VerhaeltnisReferenzertragErtrag10Jahre         22 non-null     float64       
 6   VerhaeltnisReferenzertragErtrag15Jahre         22 non-null     float64       
 7   Zuschlagsnummer                                2416 non-null   object        
 8   AnlageBetriebsstatus                           32416 non

In [135]:
### Learnings from df_bids_sum
# Seldomly all bid-nrs of a bid-date can be linked to the eeg-table
# Very bad quota linked bid-nrs / all bid-nrs for earliest bids (2017) and latest bids (2022-2023)
# Quota can not be enhanced by using the mastr or anlagen-nr

### Summary Zuschlagsdaten
print(len(df_bids_all))                                                             # 4418 Bids extracted 
print(len(df_bids_all["Zuschlagsnummer"].unique()))                                 # 2059 Unique bid nrs 
print(len(df_bids_all[df_bids_all["ZS_nr_in_eeg"]]["Zuschlagsnummer"].unique()))    # 1197 Bid nrs in eeg 
print(df_bids_all["ZS_nr_in_eeg"].sum())                                            # 2381 Units linked to eeg by bid nr

print(df_bids_all["Anl_nr_in_eeg"].sum())                                           # 1208 Units linked to eeg by anlagen_nr
print(df_bids_all["Mastr_nr_in_eeg"].sum())                                         # 1605 Units linked to eeg by mastr_nr

### All entries of df_eeg have an inbetriebnahmedatum -> explains why latest units can from bid date can not be linked (~530 of 862 bid nrs not linked)
### 856 units from anlagen_eeg build later than 2017-05-01 which can not be linked via the Zuschlagsnr to df_bids_all. Quite comparable.
### Only 70 of those have Zuschlagsnr. But a lot of these have a Scheme of the Zuschlagsnr not matching the (BK6...) Scheme of the Zuschlagsnr in df_bids_all (WIN%Y)

4418
2059
1197
2381
1208
1605
1197


In [150]:
# How do the bid-nrs not found look like -> Sort unique by date
df_bids_not_eeg = df_bids_all[~df_bids_all["ZS_nr_in_eeg"]]
df_bids_not_eeg = df_bids_not_eeg[["Gebotsdatum", "Zuschlagsnummer", "Zuschlags-Nr"]].drop_duplicates().sort_values("Gebotsdatum")

In [166]:
df_eeg_not_bids = df_eeg[~df_eeg["Zuschlagsnummer"].isin(df_bids_all["Zuschlagsnummer"])]
later_date = pd.to_datetime("2017-05-01")
df_eeg_not_bids = df_eeg_not_bids[df_eeg_not_bids["EegInbetriebnahmedatum"] > later_date]
# df_eeg_not_bids["Inbetriebmonat"] = df_eeg_not_bids["EegInbetriebnahmedatum"].dt.strftime("%Y-%m")
df_eeg_not_bids = df_eeg_not_bids[["EegInbetriebnahmedatum", "Zuschlagsnummer"]].drop_duplicates().sort_values("EegInbetriebnahmedatum")

In [168]:
df_eeg_not_bids

<class 'pandas.core.frame.DataFrame'>
Index: 856 entries, 21571 to 32412
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EegInbetriebnahmedatum  856 non-null    datetime64[ns]
 1   Zuschlagsnummer         70 non-null     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 20.1+ KB
